In [2]:
import gradio as gr
import tensorflow as tf
import cv2
import numpy as np

In [11]:
model = tf.keras.models.load_model('best_model.keras')

In [12]:
class_names = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

In [13]:
# Hàm xử lý từng khung hình và nhận dạng hoa
def process_frame(frame):
    # Chuyển đổi khung hình về kích thước mà mô hình yêu cầu (64x64)
    resized_frame = cv2.resize(frame, (64, 64))
    # Chuyển đổi khung hình về định dạng mà mô hình yêu cầu
    normalized_frame = resized_frame / 255.0
    # Mở rộng chiều để phù hợp với input_shape của mô hình
    expanded_frame = np.expand_dims(normalized_frame, axis=0)
    # Dự đoán nhãn
    predictions = model.predict(expanded_frame)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    # Trả về tên loại hoa
    return class_names[predicted_class]

In [14]:
# Hàm nhận dạng hoa trong video và trả về danh sách thời gian và nhãn hoa
def recognize_flowers_in_video(video):
    cap = cv2.VideoCapture(video)
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Lấy thời gian hiện tại trong video
        time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # thời gian tính bằng giây
        # Nhận dạng hoa trên khung hình
        flower_name = process_frame(frame)
        # Lưu kết quả với thời gian tương ứng
        results.append((time, flower_name))

    cap.release()
    return results

In [15]:
# Hàm Gradio để xử lý video và hiển thị kết quả
def gradio_video_processing(video):
    results = recognize_flowers_in_video(video)
    # Tạo danh sách hiển thị kết quả với tên loại hoa
    display_results = [{"Time (s)": time, "Flower Name": flower_name} for time, flower_name in results]
    return display_results

In [16]:
# Tạo giao diện Gradio
interface = gr.Interface(
    fn=gradio_video_processing,
    inputs=gr.Video(),
    outputs=gr.JSON(),
    title="Flower Recognition from Video",
    description="Upload a video to recognize flowers and get the results with timestamps."
)

In [17]:
# Chạy ứng dụng Gradio
interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Phan Tuan\anaconda3\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Phan Tuan\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━